In [0]:
import utils
import pandas as pd
import numpy as np
from sklearn.externals import joblib

import category_encoders as ce

import importlib
importlib.reload(utils)

pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', -1)  # or 199

# Get data

In [0]:
df1=pd.read_csv('/home/gjain/data_000000000000.csv')
df2=pd.read_csv('/home/gjain/data_000000000001.csv')
df3=pd.read_csv('/home/gjain/data_000000000002.csv')

df=df1.append(df2)
df=df.append(df3)

# df=pd.read_csv('/home/gjain/dec_data_000000000000.csv')

df.drop_duplicates(subset='order_number', keep='first', inplace=True)
print (df.shape)

df.target.value_counts()

(1389202, 91)


0    1386632
1    2570   
Name: target, dtype: int64

# Feature engineering

In [0]:
email=df['customer_email'].str.split("@",expand=True)
email.columns=['id','domain']
df['email_id']=email['id']
df['email_user_id_len']=email['id'].astype('str').apply(lambda x: len(x) if x is not None else 0)
df['email_domain']=email['domain']

#DOB splits
dob=df['dob'].astype('str').str.split(" ",expand=True)
dob=dob[dob.columns[0]].str.split("-",expand=True)
dob.columns=['year','month','day']
df['dob_year']=dob['year'].astype('str')
df['dob_month']=dob['month'].astype('str')
df['dob_day']=dob['day'].astype('str')

df['bookerIsTravelAgency'] = df.customer_name.astype('str').apply(lambda x: utils.booker_is_travel_agency(x) if x is not None else False)
df['successful_attempt_no']=df.success_attempt_id.apply(lambda x: utils.attempt_no(x))
df['country_ip_flight_MatchScore']=df.apply(utils.country_ip_flight_match_score, axis=1)
df['flight_distance']=df.apply(utils.one_way_distance, axis=1)
df['days_to_departure']=df.apply(utils.days_to_departure,axis=1)
df['vacation_length']=df.apply(utils.vacation_length,axis=1)
df['cities_in_itinerary']=df.Itinerary.astype('str').apply(lambda x: utils.cities_in_itinerary(x) if x is not None else 0)
df['cabin_class']=df.CabinClasses.astype('str').apply(lambda x: utils.cabinclass(x))
df['bookerAge']=df.dob_year.apply(lambda x: utils.get_age(x))
df['name_len']=df.customer_name.astype('str').apply(lambda x: len(x) if x is not None else 0)

## Time related variables to be binned
df['weekend_booking']=df.OrderDateBrandTime_DayOfWeek.apply(lambda x: 1 if x in (1,7) else 0)
df['booking_daytime']= df.OrderDateBrandTime_Hour.apply(lambda x: utils.booking_daytime(x))
df['bookerAgeBracket']=df.bookerAge.apply(lambda x: utils.booker_age_bracket(x))
df['channelcode'].fillna('Direct',inplace=True)

df['dob_month'].fillna(0.0,inplace=True)

df.customer_name.fillna('Unknown',inplace=True)
df.email_id.fillna(value='Unknown',inplace=True)

df['eci']=df['eci'].apply(lambda x: x.replace('0','',1) if isinstance(x,str) else str(x).replace('0','',1))

#This can throw error if customer name or email id is nan
score = np.vectorize(utils.jaro_similarity, otypes=[np.float])(df.customer_name.values, df.email_id.values)
vector_emailed2 = np.vectorize(utils.email_real_check2)
email_sanity_score=vector_emailed2(df.customer_name.values, df.email_id.values, df.dob_year.values, df.dob_month.values,
                                   df.dob_day.values, score)
df['email_sanity_score']=email_sanity_score

#Cyclical features from timestamp
df['ordermonth_sin']=df.OrderDateBrandTime_Month.apply(lambda x: utils.encode_circular(x,"sin", 12))
df['ordermonth_cos']=df.OrderDateBrandTime_Month.apply(lambda x: utils.encode_circular(x,"cos", 12))

dd=df.DepartureDate.apply(lambda x: pd.to_datetime(x).date().month)
df['departuremonth_sin']=dd.apply(lambda x: utils.encode_circular(x,"sin", 12))
df['departuremonth_cos']=dd.apply(lambda x: utils.encode_circular(x,"cos", 12))

######################################################################################
cols_to_drop=['BookerContactPhone','success_attempt_id','email_id','customer_email','dob','card_num','card_country','BookerStreet','BookerZipCode','BookerState','card_bin','customer_name','OriginLatitude','OriginLongitude','DestinationLatitude','DestinationLongitude',
             'DepartureDate','ReturnDate','EndDate','FareType','GdsOfficeId','Itinerary','BookingClasses','CabinClasses','TicketingMethod','GA_DeviceIsMobile','GA_DeviceCategory','OrderDateBrandTime_Year',
             'ip_address','dob_day','dob_year',
              'OrderDateBrandTime_Month',
              'OrderDateBrandTime_Hour','bookerAge','BrandTimeZone']

df.drop(cols_to_drop,axis=1,inplace=True)

# Missing value treatment

In [0]:
df.fillna('Unknown',axis=1,inplace=True)

In [0]:
!pip install category_encoders==1.2.8

     |████████████████████████████████| 51kB 3.7MB/s eta 0:00:011
     |████████████████████████████████| 6.2MB 14.1MB/s eta 0:00:01
  Found existing installation: statsmodels 0.9.0
    Uninstalling statsmodels-0.9.0:
      Successfully uninstalled statsmodels-0.9.0
  Found existing installation: category-encoders 2.0.0
    Uninstalling category-encoders-2.0.0:
      Successfully uninstalled category-encoders-2.0.0
You should consider upgrading via the 'pip install --upgrade pip' command.


# Categorical encoding

In [0]:
all_cat_cols=['affiliate','channelcode','trans_currency','nonmor','payment_method','eci','card_cvvmatch','card_avsmatch','customer_title','website_language',
          'brand_continent','HasTicketInsurance','HasInsurance','HasBaggage','HasBaggage_Sale','HasAirhelpPlus','UsedVoucher','HasSP','gender',
          'BookerCity','BookerCountry_Name','airline','Consolidator','OriginAirportCode','OriginCityName','OriginCountryName','OriginWorldPartName','OriginRegion',
          'OriginSubRegion','DestinationAirportCode','DestinationCityName','DestinationCountryName','DestinationWorldPartName','DestinationRegion','DestinationSubRegion',
          'Supplier','DomesticOrInternational','Haul','OneWayOrReturn','IsCombinedOneWay','IsITFare',
          'HasSplitPnr','DeviceType','IsLowCost','email_domain','cabin_class','bookerIsTravelAgency','weekend_booking',
#           'OrderDateBrandTime_Month',
          'dob_month','booking_daytime',
          'bookerAgeBracket',
           #new add
          'OrderDateBrandTime_DayOfWeek',
          #boolean cols
          'nonmor', 'HasTicketInsurance', 'HasInsurance', 'HasBaggage', 'HasBaggage_Sale', 'HasAirhelpPlus', 'UsedVoucher', 'HasSP',
          'IsCombinedOneWay', 'IsITFare', 'HasSplitPnr', 'IsLowCost', 'bookerIsTravelAgency']

encoder = ce.OrdinalEncoder(cols=all_cat_cols, drop_invariant=False, return_df=True, handle_unknown='impute')
encoder.fit(df)
df = encoder.transform(df)

joblib.dump(encoder, '/home/gjain/ordinal_encoder.pkl', compress=3)

df.isnull().sum()
# df[df.isin([np.nan, np.inf, -np.inf]).any(1)].head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [0]:
importlib.reload(ce)
ce.__version__

'2.0.0'

In [0]:
df.target.value_counts()

0    1386632
1    2570   
Name: target, dtype: int64

In [0]:
df.to_pickle('/home/gjain/df_cat_enc_part1.pkl')